## Project Workflow 

1. Dataset - Load the dataset, Preprocessing, Data cleaning, Handling imbalance in the data, Feature Building 
2. Exploratory Data Analysis - All sorts of plots, Geological plots for better analysis, Correlation plots between different features 
3. Model Building - Designing the model, Building the model and working on metrics of the model. 

### Dataset

In [29]:
!pip install --upgrade imbalanced-learn
!pip install --upgrade scikit-learn

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 


from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder

from imblearn.over_sampling import SMOTE
import imblearn
import sklearn

print("imbalanced-learn version:", imblearn.__version__)
print("scikit-learn version:", sklearn.__version__)


ImportError: cannot import name '_check_X' from 'imblearn.utils._validation' (C:\Users\akshg\anaconda3\Lib\site-packages\imblearn\utils\_validation.py)

In [ ]:
df = pd.read_csv('Transactiondata.csv')
df.head()

### Exploratory Data Analysis

In [ ]:
df.info()

In [ ]:
## Data preproceesing 
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['Amount'], bins=30, kde=True)
plt.title('Distribution of Transaction Amount')
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
sns.countplot(x='Agent_Type', hue='Is_Fraud', data=df)
plt.title('Distribution of Fraud Transactions by Agent Type')
plt.show()

In [ ]:
df['Date_Of_Txn'] = pd.to_datetime(df['Date_Of_Txn'], errors='coerce')
df.set_index('Date_Of_Txn', inplace=True)
df.resample('D').size().plot(title='Number of Transactions per Day', figsize=(12, 6))
plt.show()

In [ ]:
numeric_columns = df.select_dtypes(include='number').columns
correlation_matrix = df[numeric_columns].corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

### Model Building

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [ ]:
df_2 = df.drop(['Transaction_ID', 'Sender_Name', 'Receiver_Name'], axis = 1)

In [ ]:
df_2 = pd.get_dummies(df_2)

In [ ]:
X = df_2.drop(['Is_Fraud'], axis = 1)
y = df_2['Is_Fraud']

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

In [22]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [26]:
prediction = model.predict(X_test)
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))

[[2932    8]
 [ 325 2599]]
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      2940
           1       1.00      0.89      0.94      2924

    accuracy                           0.94      5864
   macro avg       0.95      0.94      0.94      5864
weighted avg       0.95      0.94      0.94      5864

